In [ ]:
# Import Packages
import time
import socket
import paramiko
import pandas as pd
from pathlib import Path
from getpass import getpass

# SSH Connection Configuration
ssh_port = 22
use_enable = True

In [ ]:
# Input username and password (credential) for remote access to switch
username =  input("Masukkan username switch : ")
password = getpass("Masukkan password switch : ")
enable_secret = getpass("Masukkan enable secret switch : ")

In [ ]:
# Function for configuring
def ssh_commands(ip, port, username, password, commands, use_enable, enable_secret=''):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    try:
        print(f"\n🔹 Menghubungkan ke switch {ip} ...")        
        client.connect(ip, port=port, username=username, password=password, 
                       look_for_keys=False, allow_agent=False, timeout=10)
        
        remote_connection = client.invoke_shell()
        time.sleep(2)
        remote_connection.recv(65535)  # Flush buffer        
        print(f"✅ Koneksi ke {ip} berhasil!")

         # Mengirimkan perintah ke switch
        if use_enable:
            print(f"\n🔹 Masuk ke mode enable ...")
            remote_connection.send('enable\n')
            time.sleep(2)
            remote_connection.send(enable_secret + '\n')
            time.sleep(2)
            remote_connection.recv(65535)
            print(f"✅ Berhasil masuk ke mode enable!")
    
        for command in commands:
            print(f"🔄 Menjalankan perintah: {command}")
            remote_connection.send(command + '\n')
            time.sleep(2)
            output = remote_connection.recv(65535).decode('ascii', errors='ignore')
            print(output)

    except (paramiko.ssh_exception.NoValidConnectionsError, socket.timeout):
        print(f"❌ Gagal terhubung ke {ip}: Port tidak terbuka atau timeout.")

    except paramiko.AuthenticationException:
        print(f"❌ Autentikasi gagal ke {ip}: Periksa username/password.")

    except Exception as e:
        print(f"❌ Terjadi kesalahan pada {ip}: {e}")

    finally:
        client.close()
        print(f"🔻 Koneksi ke {ip} ditutup.\n")

In [ ]:
# Read switch configuration map file
file_name = 'switch_config_map.xlsx'
switch_merk = 'cisco'

file_path = Path.cwd()
file_path = rf'{file_path}\data\unit_A\{switch_merk}\{file_name}'
df_switch = pd.read_excel(file_path)
df_switch

In [ ]:
# Executing scripts for configuring
for _, row in df_switch.iterrows():
    ip = row["IP"]
    command_row = "Commands_1"
    commands = row[command_row].split('\n') if pd.notna(row[command_row]) else []

    if not commands:
        print(f"⚠️ Tidak ada perintah untuk {ip}, melewati...")
        continue

    ssh_commands(ip=ip, port=ssh_port, username=username, password=password, commands=commands, use_enable=use_enable, enable_secret=enable_secret)
    # break